In [26]:
## Focused analysis of FromSoftware games ##
# https://github.com/jbloewencolon/Steam-Game-Review-Sentiment-Analysis/tree/main
import pandas as pd
import re
import numpy as np
#import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, StandardScaler
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import randint, uniform
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from textblob import TextBlob, Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.phrases import Phrases
from collections import Counter
import random

import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'nltk'

In [9]:
df = pd.read_csv('data/FromSoft_reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333982 entries, 0 to 333981
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      333982 non-null  int64  
 1   query_summary                   0 non-null       float64
 2   cursors                         0 non-null       float64
 3   recommendationid                333947 non-null  float64
 4   language                        333947 non-null  object 
 5   review                          333118 non-null  object 
 6   timestamp_created               333947 non-null  float64
 7   timestamp_updated               333947 non-null  float64
 8   voted_up                        333947 non-null  object 
 9   votes_up                        333947 non-null  float64
 10  votes_funny                     333947 non-null  float64
 11  weighted_vote_score             333947 non-null  float64
 12  comment_count   

C:\Users\brynj\AppData\Local\Temp\ipykernel_3292\1699989045.py:1: DtypeWarning: Columns (8,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/FromSoft_reviews.csv')


In [10]:
#Drop Nulls
df.dropna(subset=['review'], inplace=True)

#Keep only English reviews
df = df[df['language'] == 'english']

# Drop Unnecessary Columns
#df = df.drop(df.columns[[0, 1, 2, 6, 7, 15, 16, 17]], axis=1)

# Create a mask where each review has more than one word and at least one alphabetic character
mask = df['review'].apply(lambda x: len(re.findall(r'\b\w+\b', str(x))) > 5 and bool(re.search('[a-zA-Z]', str(x))))

# Apply the mask to the DataFrame to filter out reviews
df = df[mask]

In [11]:
# More thank 100k empty reviews dropped
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206236 entries, 0 to 333981
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      206236 non-null  int64  
 1   query_summary                   0 non-null       float64
 2   cursors                         0 non-null       float64
 3   recommendationid                206236 non-null  float64
 4   language                        206236 non-null  object 
 5   review                          206236 non-null  object 
 6   timestamp_created               206236 non-null  float64
 7   timestamp_updated               206236 non-null  float64
 8   voted_up                        206236 non-null  object 
 9   votes_up                        206236 non-null  float64
 10  votes_funny                     206236 non-null  float64
 11  weighted_vote_score             206236 non-null  float64
 12  comment_count        

In [12]:
df['author.playtime_forever'].describe()

count    2.062360e+05
mean     8.872891e+03
std      1.706273e+04
min      5.000000e+00
25%      2.685000e+03
50%      5.202000e+03
75%      1.015700e+04
max      2.605646e+06
Name: author.playtime_forever, dtype: float64

In [13]:
# Calculate the length of each review (in words)
df['review_length'] = df['review'].apply(lambda x: len(x.split()))

# Calculate the average length of reviews
average_length = df['review_length'].mean()

df['review_length'].describe()

count    206236.000000
mean         62.836930
std         128.173951
min           1.000000
25%          10.000000
50%          21.000000
75%          57.000000
max        2396.000000
Name: review_length, dtype: float64

In [21]:
# Calculate quantiles for playtime
df['playtime_category'] = pd.qcut(df['author.playtime_forever'], 3, labels=False)

# Check value counts
print(df['playtime_category'].value_counts())

playtime_category
0    68759
2    68739
1    68738
Name: count, dtype: int64


In [22]:
# Define preprocessing for text column
text_features = 'review'
text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=2000))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features)
    ])

# Sample 75% of data
df_sampled = df.sample(frac=0.30, random_state=42)

NameError: name 'Pipeline' is not defined